# Retrieving Data from Chacolgenide Perovskites

```{admonition} Motivation 

:class:note

In this notebook we want to collect a dataset os structured data that will allow us to train a composition-property prediction model later on. But we want to focus on a specific class of materials called **chalcogenide perovskites**.

- We want to ensure that the output formulas retrieved can be parsed into a pymatgen `Composition` object. This is important to be able to reuse the extracted data with popular materials informatics packages, such as those leading in [MatBench](https://hackingmaterials.lbl.gov/matbench/).
- We want to ensure control over  the stoichiometry of the compounds retrieved. We also want to ensure that the formulas fulfill the criteria of charge neutrality. For this, we will employ a constrained decoding approach as exemplify in the [constrained decoding]() notebook.
- We will validate if the formulas retrieved fulfill some stability criteria for being a perovskite compound. For this we will employ a similar approach as in the [validation example]() notebook.
  
:::

## What are Chalcogenide Perovskites?

```{admonition} Background
:class: note

We aim to constrain our extraction pipeline to output formulas that adhere to specific chemical and structural criteria, focusing on **chalcogenide perovskite** compounds. 

Chalcogenide perovskites are a subclass of perovskites where the anions are chalcogens (**sulfur**, **selenium**, or **tellurium**). 

These materials are of great interest due to their unique properties that make them suitable for energy applications, such as:

- **Optoelectronic Properties**: They exhibit favorable band gaps and strong light absorption, making them ideal for photovoltaic applications.
- **Stability**: Compared to their halide counterparts, chalcogenide perovskites often show enhanced chemical stability.
- **Flexibility in Composition**: The ability to tune the composition allows for optimization of properties for specific applications like solar cells, photodetectors, and LEDs.
```


### The Perovskite-type Structure

The perovskite-type structure is defined based on the [mineral CaTiO₃](https://en.wikipedia.org/wiki/Calcium_titanate). To be classified as a perovskite, materials must exhibit close structural similarity to this archetype. Bretterniz and Schorr have clarified a [set of criteria](https://doi.org/10.1002/aenm.201802366) to determine whether a material should be called perovskite.[@breternitz_what_2018]

1. **Stoichiometry**: ABX₃, or at least an A🟠: B🔵: X🔴 ratio of 1:1:3.
2. **B-cation Coordination**: Octahedral (or distorted octahedra) coordination.
3. **3D Network**: The [BX₆] octahedra should form an all-corner-sharing 3D network.

![A chalcogenide perovskite structural model of BaZrS₃](BaZrS3.png){width=50% fig-alt="A chalcogenide perovskite structural model of BaZrS₃"}

## Parsing papers 

For this task, we want to retrieve the structure dataset from the review paper [Chalcogenide Perovskites: Tantalizing Prospects, Challenging Materials](https://onlinelibrary.wiley.com/doi/10.1002/adom.202101704). 

Firstly, we will use Nougat to extract the text from the paper. A detailed example of how to do this can be found in the [Nougat example notebook]().

```{warning}
Even the "small" Nougat model might be slow on a computer without GPU. 
In addition, this call will download relatively large model checkpoints.
```

In [3]:
from llmstructdata.utils import convert_pdf_to_markdown_text

text = convert_pdf_to_markdown_text(
    "paper.pdf",  model="0.1.0-base", no_skipping=True
)

downloading nougat checkpoint version 0.1.0-base to path /Users/kevinmaikjablonka/.cache/torch/hub/nougat-0.1.0-base


config.json: 100%|██████████| 560/560 [00:00<00:00, 450kb/s]
pytorch_model.bin: 100%|██████████| 1.31G/1.31G [01:25<00:00, 16.5Mb/s]
special_tokens_map.json: 100%|██████████| 96.0/96.0 [00:00<00:00, 510kb/s]
tokenizer.json: 100%|██████████| 2.04M/2.04M [00:00<00:00, 16.0Mb/s]
tokenizer_config.json: 100%|██████████| 106/106 [00:00<00:00, 1.34Mb/s]
/Users/kevinmaikjablonka/miniconda3/envs/dataextract2/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0%|          | 0/27 [00:07<?, ?it/s]
Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/dataextract2/bin/nougat", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/dataextract2/lib/p

ValueError: No markdown files found in the output directory.

## Constraining the formulas to have a 1:1:3 stoichiometry and charge neutrality

We will use [pymatgen](https://pymatgen.org/) to generate a list of ternary chalcogenide perovskites that fulfill at least the first criterion for sulfide and selenide compounds (the 1:1:3 stoichiometry one). 
The [`oxi_state_guesses` method](https://pymatgen.org/pymatgen.core.html#pymatgen.core.composition.Composition.oxi_state_guesses) of the pymatgen `Composition` object will ensure the stoichiometry is correct and that the compound is charge balanced. This is performed by using guessed oxidation states for the elemental combinations screened.

In [1]:
from pymatgen.core import Composition


def generate_compositions(element_symbols, anions=["S", "Se"]):
    valid_compositions = []

    for i, cation1 in enumerate(element_symbols):
        for cation2 in element_symbols[i + 1 :]:  # Ensure unique pairs
            if cation1 != cation2:  # Ensure that two cations are not the same
                for anion in anions:
                    formula = f"{cation1}{cation2}{anion}3"
                    try:
                        comp = Composition(formula)
                        # Guess oxidation states
                        oxi_states_override = {anion: [-2]}
                        oxi_state_guesses = comp.oxi_state_guesses(
                            oxi_states_override=oxi_states_override
                        )

                        for guess in oxi_state_guesses:
                            if all(val is not None for val in guess.values()):
                                valid_compositions.append((comp.reduced_formula, guess))
                                break
                    except ValueError:
                        continue  # Skip invalid combinations

    return valid_compositions


element_symbols = [
    "Sr",
    "Ba",
    "Hf",
    "Zr",
    "Eu",
    "Ti",
    "La",
    "Ce",
    "Sm",
    "Gd",
    "Tb",
    "Dy",
    "Ho",
    "Er",
    "Tm",
    "Yb",
    "Lu",
]
valid_compositions = generate_compositions(element_symbols)

for formula, oxi_states in valid_compositions:
    print(f"Formula: {formula}, Oxidation States: {oxi_states}")

print(f"Number of valid compositions: {len(valid_compositions)}")

valid_formulas = [formula for formula, _ in valid_compositions]

Formula: SrHfS3, Oxidation States: {'Sr': 2.0, 'Hf': 4.0, 'S': -2.0}
Formula: SrHfSe3, Oxidation States: {'Sr': 2.0, 'Hf': 4.0, 'Se': -2.0}
Formula: SrZrS3, Oxidation States: {'Sr': 2.0, 'Zr': 4.0, 'S': -2.0}
Formula: SrZrSe3, Oxidation States: {'Sr': 2.0, 'Zr': 4.0, 'Se': -2.0}
Formula: SrTiS3, Oxidation States: {'Sr': 2.0, 'Ti': 4.0, 'S': -2.0}
Formula: SrTiSe3, Oxidation States: {'Sr': 2.0, 'Ti': 4.0, 'Se': -2.0}
Formula: SrCeS3, Oxidation States: {'Sr': 2.0, 'Ce': 4.0, 'S': -2.0}
Formula: SrCeSe3, Oxidation States: {'Sr': 2.0, 'Ce': 4.0, 'Se': -2.0}
Formula: SrTbS3, Oxidation States: {'Sr': 2.0, 'Tb': 4.0, 'S': -2.0}
Formula: SrTbSe3, Oxidation States: {'Sr': 2.0, 'Tb': 4.0, 'Se': -2.0}
Formula: BaHfS3, Oxidation States: {'Ba': 2.0, 'Hf': 4.0, 'S': -2.0}
Formula: BaHfSe3, Oxidation States: {'Ba': 2.0, 'Hf': 4.0, 'Se': -2.0}
Formula: BaZrS3, Oxidation States: {'Ba': 2.0, 'Zr': 4.0, 'S': -2.0}
Formula: BaZrSe3, Oxidation States: {'Ba': 2.0, 'Zr': 4.0, 'Se': -2.0}
Formula: BaTiS3, Oxi

To enable constrained decoding, we will use one of the most popular packages for this task [`instructor`](https://jxnl.github.io/instructor/).
It is built on [`pydantic`]() and can leverage function calling and JSON-mode of the OpenAI API as well as other constrained sampling approaches.
This part of the notebook is based on the example provided in the [constrained decoding example notebook](index.ipynb).

In [2]:
from pydantic import BaseModel, Field
from typing import Optional, Literal
from litellm import completion
import instructor
from dotenv import load_dotenv

_ = load_dotenv("../../.env", override=True)

## Defining a data schema

For most constrained generation tasks, we need to define a data schema in a programmatic way.
The most common way to do so is to use `pydantic` data classes. For more information on how to define data classes, see the [pydantic documentation](https://pydantic-docs.helpmanual.io/) and our notebook on [constrained decoding](index.ipynb).

If we want to extract valid composition for chalcogenide perovskites, and additional properties reported in the literature, associated with them. In our schema, we will only allow compositions from our precompiled list.

We can now use `instructor` to "patch" the OpenAI API client to ensure that our output fulfills the schema.

In [3]:
client = instructor.from_litellm(completion)
model = "gpt-3.5-turbo-0125"


# Create the model
class ChalcogenidePerovskite(BaseModel):
    name: str = Field(
        ..., title="Name", description="Name of the chalcogenide perovskite."
    )
    bandgap: Optional[float] = Field(
        None,
        title="Bandgap",
        description="Bandgap of the chalcogenide perovskite. Must be greater or equal to 0.",
        ge=0,
    )
    formula: Optional[Literal[tuple(valid_formulas)]] = Field(
        None,
        title="Formula",
        description="Chemical formula of the chalcogenide perovskite.",
        pattern=r"^([A-Z][a-z]?){3}3$",
    )
    synthesis_method: Optional[str] = Field(
        None,
        title="Synthesis Method",
        description="Synthesis method of the chalcogenide perovskite.",
    )


# Example usage
example = ChalcogenidePerovskite(
    name="Example",
    bandgap=1.5,
    formula="BaZrS3",
    synthesis_method="Solid-state reaction",
)

print(example.dict())

{'name': 'Example', 'bandgap': 1.5, 'formula': 'BaZrS3', 'synthesis_method': 'Solid-state reaction'}


In [4]:
messages = [
    {
        "role": "system",
        "content": "You are an expert chemist and your work is to extract information about perovskites from the text provided according to the schema.",
    },
    {
        "role": "user",
        "content": "Extract the information from the following text:\n\n {text}",  # This text variable should contain the chunks of the article after Nougat extract them
    },
]

In [5]:
constrained_answer = client.chat.completions.create(
    model=model,
    response_model=ChalcogenidePerovskite,
    temperature=0,
    max_retries=2,
    caching=True,
    messages=messages,
)

## Extracting the data for 1:1:3 compounds

## Validating the output to be a perovskite type structure

Before, we have ensured that the output formulas follow **the stoichiometry criteria of 1:1:3** for a perovskite-type compound. 
Now, we aim to validate whether a given set of chemical formulas can fulfill the other criteria:

1. **B-cation Coordination**: Octahedral (or distorted octahedra) coordination.
2. **3D Network**: The [BX₆] octahedra should form an all-corner-sharing 3D network.

This is something that is typically not reported in the text of the papers. However, we can calculate **the tolerance factor** that gives us a proxy about whether a perovskite structure is stable for a given composition and used this as **validation** criteria in our extraction pipeline.

```{admonition} The Tolerance and Octahedral Factors
:class: tip

**Tolerance Factor \(*t*\)**: This is a dimensionless number calculated from the ionic radii of the constituent ions. For a stable **oxide perovskite-type** structure, the tolerance factor should ideally be in the range of 0.8 to 1.0.

$$
t = \frac{r_A + r_O}{\sqrt{2} (r_B + r_O)}
$$

where *r~A~*, *r~B~*, and *r~O~* are the ionic radii of the A-site cation, B-site cation, and oxygen anion, respectively.

Many modifications to the tolerance factor formula have been proposed. We will use a modified formula that incorporates electronegativity differences in our validation process, specifically for chacolgenide perovskites as reported by Jess et al. [@jess_phase_2022]

```

To determine the tolerance factor and octahedral factor, we perform the following steps:

1. **Find Cations and Oxidation States**: We identify the A-site and B-site cations and their oxidation states for a given 1:1:3 compound formula.
2. **Fetch Elemental Properties**: We retrieve ionic radii and electronegativities of the elements involved using the `pymatgen` and `mendeleev` libraries.
3. **Calculate Modified Tolerance Factor**: We use a modified formula for the tolerance factor that incorporates electronegativity differences:
   
   $$ 
   t^* = \frac{\left( \frac{\Delta \chi_{(A-X)}}{\Delta \chi_{(A-O)}} \right) (r_A + r_X)}{\sqrt{2} \left( \frac{\Delta \chi_{(B-X)}}{\Delta \chi_{(B-O)}} \right) (r_B + r_X)} 
   $$

4. **Validation**: We validate the structure by checking if the calculated tolerance factor fall within the acceptable ranges for sulfide and selenide compounds.


Let's start by fetching the shannon ionic radii of the species involved in the perovskite structure. And store this in our working dataframe.

Get a dicctionary for the elements of the formula and guess the oxidation states. 

In [7]:
# function that given a formula returns the element symbol and the charge for its elements into a dict with key as the element symbol and value as the charge
def get_oxidation_states(formula):
    comp = Composition(formula)
    guesses = comp.oxi_state_guesses(oxi_states_override={"S": [-2], "Se": [-2]})
    return guesses[0]


# test the function
dict_oxi = get_oxidation_states("ZrBaS3")
print(dict_oxi)

{'Zr': 4.0, 'Ba': 2.0, 'S': -2.0}


Let's make sure to order the elements so they follow the A, B, X cations correcly, with A being the largest cation, B the transition metal, and X the anion. 

In [8]:
def order_oxidation_states(oxidation_states):
    cations = {el: ox for el, ox in oxidation_states.items() if ox > 0}
    anion = {el: ox for el, ox in oxidation_states.items() if ox < 0}

    sorted_cations = {
        k: v for k, v in sorted(cations.items(), key=lambda item: item[1])
    }

    return {**sorted_cations, **anion}


dict_oxi = order_oxidation_states(dict_oxi)
print(dict_oxi)

{'Ba': 2.0, 'Zr': 4.0, 'S': -2.0}


Here we will retrieve the ionic radii and the electronegativity for the elements in the formula using the `mendeleev` python package. 

In [10]:
from mendeleev.fetch import fetch_ionic_radii, fetch_table
import pandas as pd
import math

# Fetch the elements table to get the mapping from symbol to atomic number
elements_df = fetch_table("elements")
symbol_to_atomic_number = dict(zip(elements_df["symbol"], elements_df["atomic_number"]))


def get_ionic_radius(symbol, charge, site):
    # Get the atomic number for the given symbol
    atomic_number = symbol_to_atomic_number.get(symbol)

    if atomic_number is None:
        return None

    # Determine coordination number based on site
    if site == "A":
        cn = "XII"
    elif site == "B":
        cn = "VI"
    elif site == "X":
        cn = "VI"
    else:
        return None

    # Fetch the ionic radii table
    ionic_radii_df = fetch_ionic_radii(radius="ionic_radius")

    # Check if the determined coordination number is a valid column
    if cn not in ionic_radii_df.columns:
        return None

    # Filter the DataFrame for the specific atomic number and charge
    element_data = ionic_radii_df[
        (ionic_radii_df.index.get_level_values("atomic_number") == atomic_number)
        & (ionic_radii_df.index.get_level_values("charge") == charge)
    ]

    # Extract the ionic radius for the given coordination number
    if element_data.empty:
        return None

    ionic_radius = element_data[cn].values

    if len(ionic_radius) > 0 and pd.notna(ionic_radius[0]):
        return ionic_radius[0]
    else:
        return None


def get_electronegativity(symbol):
    elements_df = fetch_table("elements")
    en_pauling = elements_df.set_index("symbol")["en_pauling"].to_dict()
    return en_pauling.get(symbol)


print(get_ionic_radius("Zr", 4, "B"))
print(get_electronegativity("Zr"))

72.0
1.33


```{admonition} The Tabulated Shannon Radii Values

:class: warning

The Shannon ionic radii are tabulated for a variety of coordination numbers and oxidation states. 
We  use the values for the most common coordination number of 6 (octahedral) for the B cation, which is commonly found. For the A cation, we decided to fix the coordination number to 12. For some elements in a given oxidation state, Shannon radii data in the 12 coordination number might not be available. In those cases, we will not be able to calculate the tolerance factor. An alternative approach would be similar to the one proposed by {cite:t}`jess_phase_2022` where they perform a linear extrapolation to determine the ionic radii for the A cation in the 12 coordination number.
```

In [11]:
def calculate_modified_tolerance_factor(
    A_symbol, A_charge, B_symbol, B_charge, X_symbol="S", X_charge=-2
):
    r_A = get_ionic_radius(A_symbol, A_charge, "A")
    r_B = get_ionic_radius(B_symbol, B_charge, "B")
    r_X = get_ionic_radius(X_symbol, X_charge, "X")

    # Ensure radii are retrieved successfully
    if r_A is None or r_B is None or r_X is None:
        print(
            f"Failed to retrieve ionic radii for: A({A_symbol}, {A_charge}), B({B_symbol}, {B_charge}), X({X_symbol}, {X_charge})"
        )
        return None

    chi_A_X = get_electronegativity(A_symbol) - get_electronegativity(X_symbol)
    chi_A_O = get_electronegativity(A_symbol) - get_electronegativity("O")
    chi_B_X = get_electronegativity(B_symbol) - get_electronegativity(X_symbol)
    chi_B_O = get_electronegativity(B_symbol) - get_electronegativity("O")

    # Ensure electronegativities are retrieved successfully
    if chi_A_X is None or chi_A_O is None or chi_B_X is None or chi_B_O is None:
        print(
            f"Failed to retrieve electronegativity for: A({A_symbol}), B({B_symbol}), X({X_symbol})"
        )
        return None

    t_star = (chi_A_X / chi_A_O * (r_A + r_X)) / (
        math.sqrt(2) * chi_B_X / chi_B_O * (r_B + r_X)
    )
    return t_star


def calculate_modified_tolerance_factor_from_formula(formula):
    oxidation_states = get_oxidation_states(formula)
    oxidation_states = order_oxidation_states(oxidation_states)

    A_symbol, A_charge = list(oxidation_states.items())[0]
    B_symbol, B_charge = list(oxidation_states.items())[1]
    X_symbol, X_charge = list(oxidation_states.items())[2]

    t_star = calculate_modified_tolerance_factor(
        A_symbol, A_charge, B_symbol, B_charge, X_symbol, X_charge
    )
    return t_star


# Test the function

formula = "BaZrS3"
t_star = calculate_modified_tolerance_factor_from_formula(formula)
print(f"The tolerance factor* for {formula} is {t_star}")

The tolerance factor* for BaZrS3 is 1.0660635594441457


In [12]:
def is_perovskite(formula):
    t_star = calculate_modified_tolerance_factor_from_formula(formula)

    if t_star is None:
        return False

    # Define the boundaries for sulfides and selenides
    if "S" in Composition(formula).elements:
        return 0.9 <= t_star <= 1.09
    elif "Se" in Composition(formula).elements:
        return 1.01 <= t_star <= 1.06
    else:
        return 0.8 <= t_star <= 1.1


# Example usage
composition_str = "BaZrS3"
is_perovskite_result = is_perovskite(composition_str)
print(f"Is {composition_str} a perovskite? {is_perovskite_result}")

Is BaZrS3 a perovskite? True


In [13]:
# test for formulas in valid_compositions building a dataframe

formulas = [formula for formula, _ in valid_compositions]
resultd_df = pd.DataFrame(
    {
        "Formula": formulas,
        "Is Perovskite": [is_perovskite(formula) for formula in formulas],
        "Tolerance Factor*": [
            calculate_modified_tolerance_factor_from_formula(formula)
            for formula in formulas
        ],
    }
)
resultd_df

Failed to retrieve ionic radii for: A(Zr, 2.0), B(Hf, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(Zr, 2.0), B(Hf, 4.0), X(Se, -2.0)
Failed to retrieve ionic radii for: A(Eu, 2.0), B(Hf, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(Eu, 2.0), B(Hf, 4.0), X(Se, -2.0)
Failed to retrieve ionic radii for: A(Ti, 2.0), B(Hf, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(Ti, 2.0), B(Hf, 4.0), X(Se, -2.0)
Failed to retrieve ionic radii for: A(La, 2.0), B(Hf, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(La, 2.0), B(Hf, 4.0), X(Se, -2.0)
Failed to retrieve ionic radii for: A(Sm, 2.0), B(Hf, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(Sm, 2.0), B(Hf, 4.0), X(Se, -2.0)
Failed to retrieve ionic radii for: A(Yb, 2.0), B(Hf, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(Yb, 2.0), B(Hf, 4.0), X(Se, -2.0)
Failed to retrieve ionic radii for: A(Eu, 2.0), B(Zr, 4.0), X(S, -2.0)
Failed to retrieve ionic radii for: A(Eu, 2.0), B(Zr, 4.0), X(Se, -2.0)

,Formula,Is Perovskite,Tolerance Factor*
0,SrHfS3,True,0.995430
1,SrHfSe3,True,0.988970
2,SrZrS3,True,1.001105
3,SrZrSe3,True,0.995384
4,SrTiS3,False,1.134458
...,...,...,...
209,YbTmSe3,False,NaN
210,TmLuS3,False,NaN
211,TmLuSe3,False,NaN
212,YbLuS3,False,NaN


In [44]:
# count how many have a tolerance factor
print(resultd_df["Is Perovskite"].value_counts())

Is Perovskite
False    163
True      51
Name: count, dtype: int64


In [19]:
resultd_df["Is Perovskite"].value_counts(normalize=True)

Is Perovskite
False    0.948598
True     0.051402
Name: proportion, dtype: float64

## Extracting the data for 1:1:3 compounds that are stable in a perovskite-type structure